In [5]:
import cv2
import time
from datetime import datetime
from keras.models import load_model
from openpyxl import Workbook
from scipy.spatial import distance as dist
import numpy as np
import pandas as pd

emotion_model = load_model('emotion_model.h5')
emotion_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
# Function to calculate eye aspect ratio (EAR)
def eye_aspect_ratio(eye):
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])
    C = dist.euclidean(eye[0], eye[3])
    ear = (A + B) / (2.0 * C)
    return ear

# Initialize video capture
cap = cv2.VideoCapture(0)

# Load face and eye cascade classifiers
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')
# gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
# faces = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml').detectMultiScale(gray_frame, scaleFactor=1.3, minNeighbors=5)

# Initialize variables
start_time = time.time()
last_active_time = time.time()
distraction_threshold = 5  # in seconds
distracted = False

# Initialize Excel workbook and worksheet
wb = Workbook()
ws = wb.active
ws.append(["Timestamp", "Distracted", "Emotion", "Face Data"])

while True:
    # Read a frame from the video capture
    ret, frame = cap.read()
    if not ret:
        break

    # Convert frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    for (x, y, w, h) in faces:

        face = gray[y:y+h, x:x+w]
        face = cv2.resize(face, (48, 48))
        face = face.astype('float32') / 255.0
        face = np.expand_dims(face, axis=0)
        face = np.expand_dims(face, axis=-1)
        
        emotion_prediction = emotion_model.predict(face)
        max_index = np.argmax(emotion_prediction[0])
        emotion = emotion_labels[max_index]
        
        # Draw rectangle around detected face
        cv2.putText(frame, emotion, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)
        cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)

        # Extract region of interest (ROI) for eyes
        roi_gray = gray[y:y + h, x:x + w]
        roi_color = frame[y:y + h, x:x + w]

        # Detect eyes within the face region
        eyes = eye_cascade.detectMultiScale(roi_gray)

        # Calculate attentiveness based on eye aspect ratio (EAR)
        attentiveness = 0
        for (ex, ey, ew, eh) in eyes:
            eye = roi_gray[ey:ey + eh, ex:ex + ew]
            ear = eye_aspect_ratio(eye)
            attentiveness += ear

        # Normalize attentiveness by the number of eyes
        if len(eyes) > 0:
            attentiveness /= len(eyes)

        # Check if the user is distracted
        if attentiveness < 0.25:  # Example threshold for closed eyes
            if time.time() - last_active_time > distraction_threshold:
                distracted = True
        else:
            distracted = False
        
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        ws.append([timestamp, distracted, emotion, str(frame)])

    # Display the frame
    cv2.imshow('frame', frame)
    
    if len(faces) < 1:
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        ws.append([timestamp, True, "", "[]"])
    
    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the capture and close OpenCV windows
cap.release()
cv2.destroyAllWindows()

# Save the Excel workbook
wb.save("distraction_data.xlsx")

0
0
0
0
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 860ms/step
1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
2
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
2
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
2
0
0
0
0
0
0
0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1
0
0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1
0
0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1
0
0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1
0
0
0
0
0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1
1